# Dashboard de Vendas - Data Warehouse

Este notebook apresenta visualizações interativas dos dados do warehouse de vendas utilizando Plotly Express.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import psycopg2
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

load_dotenv(override=True)

print("Bibliotecas importadas com sucesso!")

In [ ]:
DB_CONFIG = {
    'host': 'localhost',
    'database': os.getenv('POSTGRES_DB_WAREHOUSE'),
    'user': os.getenv('POSTGRES_USER_WAREHOUSE'),
    'password': os.getenv('POSTGRES_PASSWORD_WAREHOUSE'),
    'port': os.getenv('POSTGRES_PORT_WAREHOUSE')
}

def get_connection():
    """Função para estabelecer conexão com o banco de dados"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        return conn
    except psycopg2.Error as e:
        print(f"Erro ao conectar com o banco de dados: {e}")
        return None

def execute_query(query):
    """Função para executar query e retornar DataFrame"""
    conn = get_connection()
    if conn:
        try:
            df = pd.read_sql_query(query, conn)
            conn.close()
            return df
        except Exception as e:
            print(f"Erro ao executar query: {e}")
            conn.close()
            return None
    return None

test_conn = get_connection()
if test_conn:
    print("Conexão com o banco de dados estabelecida com sucesso!")
    test_conn.close()
else:
    print("Falha ao conectar com o banco de dados.")

## 1. Total Vendido Minuto a Minuto

Visualização da evolução das vendas ao longo do tempo, agregando os valores por data e hora (minuto).

In [ ]:
query_vendas_minuto = """
SELECT 
    DATE_TRUNC('minute', v.created_at) as data_hora_minuto,
    SUM(v.valor_total) as total_vendido,
    COUNT(*) as quantidade_vendas
FROM public.venda v
WHERE v.created_at IS NOT NULL
GROUP BY DATE_TRUNC('minute', v.created_at)
ORDER BY data_hora_minuto;
"""

df_vendas_minuto = execute_query(query_vendas_minuto)

if df_vendas_minuto is not None and len(df_vendas_minuto) > 0:
    df_vendas_minuto['data_hora_minuto'] = pd.to_datetime(df_vendas_minuto['data_hora_minuto'])
    
    fig_minuto = px.line(
        df_vendas_minuto,
        x='data_hora_minuto',
        y='total_vendido',
        title='Total Vendido por Minuto',
        labels={
            'data_hora_minuto': 'Data e Hora',
            'total_vendido': 'Total Vendido (R$)'
        },
        hover_data={'quantidade_vendas': True}
    )
    
    fig_minuto.update_traces(
        line=dict(color='#2E86C1', width=2),
        hovertemplate='<b>Data/Hora:</b> %{x}<br>' +
                      '<b>Total Vendido:</b> R$ %{y:,.2f}<br>' +
                      '<b>Qtd Vendas:</b> %{customdata[0]}<extra></extra>'
    )
    
    fig_minuto.update_layout(
        title_font_size=16,
        xaxis_title_font_size=12,
        yaxis_title_font_size=12,
        height=500,
        showlegend=False
    )
    
    fig_minuto.show()
else:
    print("Nenhum dado encontrado para vendas por minuto ou erro na consulta.")

## 2. Total de Vendas por Cidade

Análise das vendas agregadas por cidade, mostrando quais cidades têm maior volume de vendas.

In [ ]:
query_vendas_cidade = """
SELECT 
    c.cidade_desc as cidade,
    c.cidade_uf as uf,
    SUM(v.valor_total) as total_vendido,
    COUNT(v.venda_id) as quantidade_vendas,
    COUNT(DISTINCT cl.cliente_id) as clientes_unicos
FROM public.venda v
JOIN public.cliente cl ON v.cliente_id = cl.cliente_id
JOIN public.cidade c ON cl.cidade_id = c.cidade_id
GROUP BY c.cidade_desc, c.cidade_uf
ORDER BY total_vendido DESC;
"""

df_vendas_cidade = execute_query(query_vendas_cidade)

if df_vendas_cidade is not None and len(df_vendas_cidade) > 0:
    fig_cidade = px.bar(
        df_vendas_cidade,
        x='cidade',
        y='total_vendido',
        color='uf',
        title='Total de Vendas por Cidade',
        labels={
            'cidade': 'Cidade',
            'total_vendido': 'Total Vendido (R$)',
            'uf': 'UF'
        },
        hover_data={
            'quantidade_vendas': True,
            'clientes_unicos': True,
            'uf': True
        }
    )
    
    fig_cidade.update_traces(
        hovertemplate='<b>Cidade:</b> %{x}<br>' +
                      '<b>UF:</b> %{customdata[2]}<br>' +
                      '<b>Total Vendido:</b> R$ %{y:,.2f}<br>' +
                      '<b>Qtd Vendas:</b> %{customdata[0]}<br>' +
                      '<b>Clientes Únicos:</b> %{customdata[1]}<extra></extra>'
    )
    
    fig_cidade.update_layout(
        title_font_size=16,
        xaxis_title_font_size=12,
        yaxis_title_font_size=12,
        height=600,
        xaxis_tickangle=-45
    )
    
    fig_cidade.show()
    
else:
    print("Nenhum dado encontrado para vendas por cidade ou erro na consulta.")

## 3. Total de Vendas por UF (Estado)

Análise das vendas agregadas por Unidade Federativa, mostrando a distribuição geográfica das vendas.

In [ ]:
query_vendas_uf = """
SELECT 
    c.cidade_uf as uf,
    SUM(v.valor_total) as total_vendido,
    COUNT(v.venda_id) as quantidade_vendas,
    COUNT(DISTINCT cl.cliente_id) as clientes_unicos,
    COUNT(DISTINCT c.cidade_desc) as cidades_uf
FROM public.venda v
JOIN public.cliente cl ON v.cliente_id = cl.cliente_id
JOIN public.cidade c ON cl.cidade_id = c.cidade_id
GROUP BY c.cidade_uf
ORDER BY total_vendido DESC;
"""

df_vendas_uf = execute_query(query_vendas_uf)

if df_vendas_uf is not None and len(df_vendas_uf) > 0:
    fig_uf = px.bar(
        df_vendas_uf,
        x='uf',
        y='total_vendido',
        title='Total de Vendas por UF (Unidade Federativa)',
        labels={
            'uf': 'Unidade Federativa (UF)',
            'total_vendido': 'Total Vendido (R$)'
        },
        hover_data={
            'quantidade_vendas': True,
            'clientes_unicos': True,
            'cidades_uf': True
        },
        color='total_vendido',
        color_continuous_scale='Blues'
    )
    
    fig_uf.update_traces(
        hovertemplate='<b>UF:</b> %{x}<br>' +
                      '<b>Total Vendido:</b> R$ %{y:,.2f}<br>' +
                      '<b>Qtd Vendas:</b> %{customdata[0]}<br>' +
                      '<b>Clientes Únicos:</b> %{customdata[1]}<br>' +
                      '<b>Cidades:</b> %{customdata[2]}<extra></extra>'
    )
    
    fig_uf.update_layout(
        title_font_size=16,
        xaxis_title_font_size=12,
        yaxis_title_font_size=12,
        height=500,
        showlegend=False
    )
    
    fig_uf.show()

else:
    print("Nenhum dado encontrado para vendas por UF ou erro na consulta.")